In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import hashlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class AthleticsDataScraper:
    def __init__(self):
        self.base_url = 'https://www.alltime-athletics.com/'
    
    def generate_url(self, event, is_legal):
        # Handle special cases with different URL patterns
        special_cases = {
            '100m': ('m_100ok.htm', 'm100mno.htm'),
            'trip': ('mtripok.htm', 'mtripno.htm'),
            'long': ('mlongok.htm', 'mlongno.htm'),
            '110h': ('m_110hok.htm', 'm_110hno.htm'),
            'pole': ('mpoleok.htm','mpoleno.htm'),
            'shot': ('mshotok.htm','mshotno.htm'),
            'disc': ('mdiscok.htm','mdiscno.htm'),
            'jave': ('mjaveok.htm','mjaveno.htm'),
            'hamm': ('mhammok.htm','mhammno.htm'),
            'deca': ('mdecaok.htm','mdecano.htm'),
            '60m':   ('m60mok.htm','m60mno.htm'),
            '800m':   ('m_800ok.htm','m_800no.htm'),
            '1500m': ('m_1500ok.htm','m_1500no.htm'),
            '5000m':   ('m_5000ok.htm','m_5000no.htm'),
            '10000':   ('m_10kok.htm','10kno.htm')
            # Add more special cases here if needed
        }
        if event in special_cases:
            legal_suffix, illegal_suffix = special_cases[event]
            suffix = legal_suffix if is_legal else illegal_suffix
        else:
            suffix = f"m_{event}{'ok' if is_legal else 'no'}.htm"
        
        return f"{self.base_url}{suffix}"

    def convert_mmss_to_seconds(self, time_str):
        parts = time_str.split(':')
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes * 60 + seconds
        else:
            return np.nan
    
    def fetch_data(self, event, is_legal):
        url = self.generate_url(event, is_legal)
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        pre_tag = soup.find('pre')
        table_text = pre_tag.get_text()
        rows = table_text.split('\n')

        def process_row(row):
            parts = re.split(r'\s{2,}', row)
            return [part.strip() for part in parts]

        data = []
        max_length = 0
        for row in rows:
            if row.strip():
                processed_row = process_row(row)
                data.append(processed_row)
                max_length = max(max_length, len(processed_row))

        # Define column names based on the maximum row length
        if max_length == 10:
            column_names = ["Test", "Rank", "Time", "Wind", "Name", "Country", "DOB", "Position_in_race", "City", "Date"]
        else:
            column_names = ["Test", "Rank", "Time", "Name", "Country", "DOB", "Position_in_race", "City", "Date"]

        df = pd.DataFrame(data, columns=column_names[:max_length])
        df.drop('Test', inplace=True, axis=1, errors='ignore')
        df['Legal'] = 'Y' if is_legal else 'N'
        has_wind = 'Wind' in df.columns
        return df, has_wind
    
    def add_all_conditions_rank(self, df, event):
        if re.search(r'\d', event):
            # This is a race event
            df['All Conditions Rank'] = df['Time'].rank(method='min')
        else:
            # This is a field event
            df['All Conditions Rank'] = df['Time'].rank(ascending=False, method='min')
        return df
    
    
    def add_age_at_time_of_race(self, df):
        # Convert DOB and Date columns to datetime with the correct format
        df['DOB'] = pd.to_datetime(df['DOB'], format='%d.%m.%Y', errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
    
        # Correct misinterpreted dates
        df['DOB'] = df['DOB'].apply(lambda x: x if pd.isnull(x) or x.year < 2023 else pd.Timestamp(year=x.year - 100, month=x.month, day=x.day))
    
        # Calculate age at the time of race
        df['Age at Time of Race'] = df.apply(lambda row: row['Date'].year - row['DOB'].year - 
                                             ((row['Date'].month, row['Date'].day) < (row['DOB'].month, row['DOB'].day)) if pd.notnull(row['DOB']) else pd.NA, axis=1)
    
        return df

    def add_competition_id(self, df):
        # Ensure 'Date' is formatted correctly
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

        # Create a concatenated string of Date and City
        df['competition_id'] = df.apply(lambda row: f"{row['Date']}_{row['City']}", axis=1)

        # Hash the concatenated string to create a unique ID
        df['competition_id'] = df['competition_id'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())

        return df
    
    def fill_mode_dob(self, df):
        # Calculate mode DOB for each athlete (Name)
        mode_dob = df.groupby('Name')['DOB'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan).reset_index()
        
        # Merge mode DOB back into original DataFrame
        df = df.merge(mode_dob, on='Name', suffixes=('', '_mode'))
        
        # Replace DOB with mode DOB where DOB is missing or different from mode DOB
        df['DOB'] = df.apply(lambda row: row['DOB_mode'] if pd.isnull(row['DOB']) or row['DOB'] != row['DOB_mode'] else row['DOB'], axis=1)
        
        # Drop temporary columns
        df.drop(columns=['DOB_mode'], inplace=True)
        
        return df

    

    def get_combined_data(self, event):
        df_legal, has_wind = self.fetch_data(event, True)

        if has_wind:
            df_illegal, _ = self.fetch_data(event, False)
            df_combined = pd.concat([df_legal, df_illegal], ignore_index=True)
        else:
            df_combined = df_legal

        df_combined.dropna(inplace=True)
        df_combined['Date'] = pd.to_datetime(df_combined['Date'], format='%d.%m.%Y')
        df_combined['DOB'] = pd.to_datetime(df_combined['DOB'], format='%d.%m.%y', errors='coerce')
        df_combined = self.fill_mode_dob(df_combined)

        # Extract any letters and '#' from the 'Time' column and put them into a new 'Note' column
        df_combined['Note'] = df_combined['Time'].str.extract(r'([a-zA-Z#*@+´]+)', expand=False)
        # Remove the letters and '#' from the 'Time' column
        df_combined['Time'] = df_combined['Time'].str.replace(r'[a-zA-Z#*@+´]', '', regex=True)
        if event in ['800','1500', '5000', '10000']:
            df_combined['Time'] = df_combined['Time'].apply(self.convert_mmss_to_seconds)
        df_combined['Time'] = df_combined['Time'].astype('float')
        df_combined['Sex'] = 'Male'
        df_combined['Event'] = event
        df_combined = self.add_all_conditions_rank(df_combined, event)

        df_combined.loc[df_combined['Legal'] == 'N', 'Rank'] = pd.NA
        df_combined = self.add_age_at_time_of_race(df_combined)

        df_combined = self.add_competition_id(df_combined)


        return df_combined


scraper = AthleticsDataScraper()
df_200m = scraper.get_combined_data('200')
df_100m = scraper.get_combined_data('100m')
df_400m = scraper.get_combined_data('400')
df_long = scraper.get_combined_data('long')
df_trip = scraper.get_combined_data('trip')
df_110h = scraper.get_combined_data('110h')
df_400h = scraper.get_combined_data('400h')
df_pole = scraper.get_combined_data('pole')
df_shot = scraper.get_combined_data('shot')
df_disc = scraper.get_combined_data('disc')
df_jave = scraper.get_combined_data('jave')
df_hamm = scraper.get_combined_data('hamm')
df_deca = scraper.get_combined_data('deca')
df_60m = scraper.get_combined_data('60m')
df_300m = scraper.get_combined_data('300')
df_800m = scraper.get_combined_data('800')
df_5000m = scraper.get_combined_data('5000')
df_10000m = scraper.get_combined_data('10000')
df_1500m = scraper.get_combined_data('1500')


In [3]:
df_10000m

,Rank,Time,Name,Country,DOB,Position_in_race,City,Date,Legal,Note,Sex,Event,All Conditions Rank,Age at Time of Race,competition_id
0,1,1571.00,Joshua Cheptegei,UGA,1996-09-12,1,Valencia,2020-10-07,Y,NaN,Male,10000,1.0,24,0422bfe2cdbecddf856ee41f164e8bf3bc33f38f
1,39,1608.36,Joshua Cheptegei,UGA,1996-09-12,1,Ad-Dawhah,2019-10-06,Y,NaN,Male,10000,39.0,23,39bbc84ed11efeb5bdf814278371fedc2cef5941
2,56,1609.94,Joshua Cheptegei,UGA,1996-09-12,2,London,2017-08-04,Y,NaN,Male,10000,56.0,20,e4ac5ae5143127a878c03d4999da472502d7eed4
3,292,1630.06,Joshua Cheptegei,UGA,1996-09-12,6,Rio de Janeiro,2016-08-13,Y,NaN,Male,10000,292.0,19,db1932a108766537b9222525868da13b72e1596f
4,479,1639.62,Joshua Cheptegei,UGA,1996-09-12,1,Gold Coast,2018-04-13,Y,NaN,Male,10000,478.0,21,ee749a0b81a3cdfbbd98574af92138f0f7241373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10056,10108,1709.95,Rui Viera,POR,1970-02-04,4rB,Barakaldo,1997-04-05,Y,NaN,Male,10000,10048.0,27,d505173d07a70fd35ebd17a2f09daa959e071dde
10057,10113,1709.96,Joe Driscoll,USA,1979-11-02,6r2,Palo Alto,2008-05-04,Y,NaN,Male,10000,10053.0,28,19e3896b4768cbf1e18a4fc70a61e70c4dbefab1
10058,10115,1709.97,Andrew Garnham,AUS,1960-02-06,9,Melbourne,1986-12-18,Y,NaN,Male,10000,10055.0,26,334cb4c6aac64fbfe1236661e65a79fe96e672da
10059,10115,1709.97,Aleksandr Burtsev,BLR,1965-02-13,4,Moskva,1990-06-09,Y,NaN,Male,10000,10055.0,25,162f22b38cf055e4619f23c494e3358a7ee96ff2


In [4]:
class AthleticsDataScraper_w:
    def __init__(self):
        self.base_url = 'https://www.alltime-athletics.com/'
    
    def generate_url(self, event, is_legal):
        # Handle special cases with different URL patterns
        special_cases = {
            '100m': ('w_100ok.htm', 'w_100no.htm'),
            'trip': ('wtripleok.htm', 'wtripleno.htm'),
            'long': ('wlongok.htm', 'wlongno.htm'),
            '100h': ('w_100hok.htm', 'w_100hno.htm'),
            'pole': ('wpoleok.htm','wpoleno.htm'),
            'shot': ('wshotok.htm','wshotno.htm'),
            'disc': ('wdiscok.htm','wdiscno.htm'),
            'jave': ('wjaveok.htm','wjaveno.htm'),
            'hamm': ('whammok.htm','whammno.htm'),
            'hept': ('whepaok.htm','wheptno.htm'),
            '60m':   ('w60mok.htm','w60mno.htm'),
            '800m':   ('w_800ok.htm','w_800no.htm'),
            '1500m': ('w_1500ok.htm','w_1500no.htm'),
            '5000m':   ('w_5000ok.htm','w_5000no.htm'),
            '10000':   ('w_10kok.htm','w_10kno.htm')
            # Add more special cases here if needed
        }
        if event in special_cases:
            legal_suffix, illegal_suffix = special_cases[event]
            suffix = legal_suffix if is_legal else illegal_suffix
        else:
            suffix = f"w_{event}{'ok' if is_legal else 'no'}.htm"
        
        return f"{self.base_url}{suffix}"
    
    def fetch_data(self, event, is_legal):
        url = self.generate_url(event, is_legal)
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        pre_tag = soup.find('pre')
        table_text = pre_tag.get_text()
        rows = table_text.split('\n')

        def process_row(row):
            parts = re.split(r'\s{2,}', row)
            return [part.strip() for part in parts]

        data = []
        max_length = 0
        for row in rows:
            if row.strip():
                processed_row = process_row(row)
                data.append(processed_row)
                max_length = max(max_length, len(processed_row))

        # Define column names based on the maximum row length
        if max_length == 10:
            column_names = ["Test", "Rank", "Time", "Wind", "Name", "Country", "DOB", "Position_in_race", "City", "Date"]
        else:
            column_names = ["Test", "Rank", "Time", "Name", "Country", "DOB", "Position_in_race", "City", "Date"]

        df = pd.DataFrame(data, columns=column_names[:max_length])
        df.drop('Test', inplace=True, axis=1, errors='ignore')
        df['Legal'] = 'Y' if is_legal else 'N'
        has_wind = 'Wind' in df.columns
        return df, has_wind
    
    def convert_mmss_to_seconds(self, time_str):
        parts = time_str.split(':')
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes * 60 + seconds
        else:
            return np.nan
        
    def add_all_conditions_rank(self, df, event):
        if re.search(r'\d', event):
            # This is a race event
            df['All Conditions Rank'] = df['Time'].rank(method='min')
        else:
            # This is a field event
            df['All Conditions Rank'] = df['Time'].rank(ascending=False, method='min')
        return df
    
    
    def add_age_at_time_of_race(self, df):
        # Convert DOB and Date columns to datetime with the correct format
        df['DOB'] = pd.to_datetime(df['DOB'], format='%d.%m.%Y', errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
    
        # Correct misinterpreted dates
        df['DOB'] = df['DOB'].apply(lambda x: x if pd.isnull(x) or x.year < 2023 else pd.Timestamp(year=x.year - 100, month=x.month, day=x.day))
    
        # Calculate age at the time of race
        df['Age at Time of Race'] = df.apply(lambda row: row['Date'].year - row['DOB'].year - 
                                             ((row['Date'].month, row['Date'].day) < (row['DOB'].month, row['DOB'].day)) if pd.notnull(row['DOB']) else pd.NA, axis=1)
        return df



    def add_competition_id(self, df):
        # Ensure 'Date' is formatted correctly
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

        # Create a concatenated string of Date and City
        df['competition_id'] = df.apply(lambda row: f"{row['Date']}_{row['City']}", axis=1)

        # Hash the concatenated string to create a unique ID
        df['competition_id'] = df['competition_id'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())

        return df
    
    def fill_mode_dob(self, df):
        # Calculate mode DOB for each athlete (Name)
        mode_dob = df.groupby('Name')['DOB'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan).reset_index()
        
        # Merge mode DOB back into original DataFrame
        df = df.merge(mode_dob, on='Name', suffixes=('', '_mode'))
        
        # Replace DOB with mode DOB where DOB is missing or different from mode DOB
        df['DOB'] = df.apply(lambda row: row['DOB_mode'] if pd.isnull(row['DOB']) or row['DOB'] != row['DOB_mode'] else row['DOB'], axis=1)
        
        # Drop temporary columns
        df.drop(columns=['DOB_mode'], inplace=True)
        
        return df

    

    def get_combined_data(self, event):
        df_legal, has_wind = self.fetch_data(event, True)

        if has_wind:
            df_illegal, _ = self.fetch_data(event, False)
            df_combined = pd.concat([df_legal, df_illegal], ignore_index=True)
        else:
            df_combined = df_legal

        df_combined.dropna(inplace=True)
        df_combined['Date'] = pd.to_datetime(df_combined['Date'], format='%d.%m.%Y',errors='coerce')
        df_combined['DOB'] = pd.to_datetime(df_combined['DOB'], format='%d.%m.%y', errors='coerce')
        df_combined = self.fill_mode_dob(df_combined)

        # Extract any letters and '#' from the 'Time' column and put them into a new 'Note' column
        df_combined['Note'] = df_combined['Time'].str.extract(r'([a-zA-Z#*@+´]+)', expand=False)
        # Remove the letters and '#' from the 'Time' column
        df_combined['Time'] = df_combined['Time'].str.replace(r'[a-zA-Z#*@+´]', '', regex=True)
        if event in ['800','1500', '5000', '10000']:
            df_combined['Time'] = df_combined['Time'].apply(self.convert_mmss_to_seconds)
        df_combined['Time'] = df_combined['Time'].astype('float')
        df_combined = self.add_all_conditions_rank(df_combined, event)

        df_combined.loc[df_combined['Legal'] == 'N', 'Rank'] = pd.NA
        df_combined = self.add_age_at_time_of_race(df_combined)
        df_combined['Sex'] = 'Female'
        df_combined['Event'] = event
        df_combined = self.add_competition_id(df_combined)
        return df_combined


scraper = AthleticsDataScraper_w()
df_200m_w = scraper.get_combined_data('200')
df_100m_w = scraper.get_combined_data('100m')
df_400m_w = scraper.get_combined_data('400')
df_long_w = scraper.get_combined_data('long')
df_trip_w = scraper.get_combined_data('trip')
df_100h_w = scraper.get_combined_data('100h')
df_400h_w = scraper.get_combined_data('400h')
df_pole_w = scraper.get_combined_data('pole')
df_shot_w = scraper.get_combined_data('shot')
df_disc_w = scraper.get_combined_data('disc')
df_jave_w = scraper.get_combined_data('jave')
df_hamm_w = scraper.get_combined_data('hamm')
df_hept_w = scraper.get_combined_data('hept')
df_60m_w = scraper.get_combined_data('60m')
df_300m_w = scraper.get_combined_data('300')
df_800m_w = scraper.get_combined_data('800')
df_5000m_w = scraper.get_combined_data('5000')
df_10000m_w = scraper.get_combined_data('10000')
df_1500m_w = scraper.get_combined_data('1500')


In [5]:
dfs_women = [df_200m_w, df_100m_w, df_400m_w,
              df_long_w, df_trip_w, df_100h_w, 
              df_400h_w, df_pole_w, df_shot_w, 
              df_disc_w, df_jave_w, df_hamm_w, 
              df_hept_w, df_60m_w, df_300m_w,
              df_1500m_w,df_800m_w, df_10000m_w,
              df_5000m_w]
combined_women = pd.concat(dfs_women, ignore_index=True)

# Combine all men's dataframes
dfs_men = [df_200m, df_100m, df_400m,
            df_long, df_trip, df_110h,
              df_400h, df_pole, df_shot,
                df_disc, df_jave, df_hamm,
                  df_deca, df_60m, df_300m,
                  df_1500m, df_5000m, df_10000m,
                  df_800m]
combined_men = pd.concat(dfs_men, ignore_index=True)

# Ensure the 'Wind' column is present in all dataframes
if 'Wind' not in combined_women.columns:
    combined_women['Wind'] = pd.NA
if 'Wind' not in combined_men.columns:
    combined_men['Wind'] = pd.NA

# Combine women's and men's dataframes
combined_all = pd.concat([combined_women, combined_men], ignore_index=True)

In [6]:
combined_all.Event.value_counts()

1500     13818
5000     13656
10000    12528
800      10665
110h     10330
100m     10117
200       8392
400       8324
pole      7826
hamm      7685
400h      6992
60m       6157
shot      4303
disc      3985
long      3370
jave      3350
trip      3152
deca      2737
100h      2420
hept      2315
300        864
Name: Event, dtype: int64

In [20]:
combined_all['Track/Field'] = combined_all['Event'].apply(lambda x: 'Track' if any(char.isdigit() for char in x) else 'Field')

In [21]:
combined_all

,Rank,Time,Wind,Name,Country,DOB,Position_in_race,City,Date,Legal,Note,All Conditions Rank,Age at Time of Race,Sex,Event,competition_id,Track/Field
0,1,21.34,+1.3,Florence Griffith-Joyner,USA,1959-12-21,1,Seoul,1988-09-29,Y,NaN,1.0,28,Female,200,457294ca525d32c99efe07c6c4bc06c44b631fbb,Track
1,7,21.56,+1.7,Florence Griffith-Joyner,USA,1959-12-21,1s1,Seoul,1988-09-29,Y,NaN,7.0,28,Female,200,457294ca525d32c99efe07c6c4bc06c44b631fbb,Track
2,31,21.76,+0.7,Florence Griffith-Joyner,USA,1959-12-21,1q1,Seoul,1988-09-28,Y,NaN,35.0,28,Female,200,6e3ff87309c0df2a9ef6f375393587dd6cbe64eb,Track
3,34,21.77,-0.1,Florence Griffith-Joyner,USA,1959-12-21,1q2,Indianapolis,1988-07-22,Y,NaN,38.0,28,Female,200,26e15d0570c9fe4a1c7e5048ec336bd9b0fd8a6c,Track
4,73,21.85,+1.3,Florence Griffith-Joyner,USA,1959-12-21,1,Indianapolis,1988-07-23,Y,NaN,82.0,28,Female,200,b2719fdb9905ce59cd71eef0858d4e9a90e98de4,Track
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142981,8311,105.98,NaN,Jinson Johnson,IND,1991-03-15,1,Bengaluru,2016-07-11,Y,NaN,8303.0,25,Male,800,9350e4033329f2cf91549501257ed19883eac389,Track
142982,8385,105.99,NaN,Ivan Nesterov,RUS,1985-02-10,2,Tula,2006-07-16,Y,NaN,8377.0,21,Male,800,8c3953bbcfa01a251c75d71159ed75abf14a60b3,Track
142983,8385,105.99,NaN,Joshua Lay,GBR,2000-04-11,1rA,Espoo,2021-06-10,Y,NaN,8377.0,21,Male,800,ec9661d026624d1e4b70a344e4eb50d8f75bcd1e,Track
142984,8385,105.99,NaN,Jose Ignacio Perez,ESP,2001-01-06,5rA,Castellón,2023-06-14,Y,NaN,8377.0,22,Male,800,f7cc345ccf2ed2e98629b7d7fbdadd23c4b51aec,Track
